In [1]:
import sys
import os

# Add the parent directory to sys.path if models.py is in the parent folder
sys.path.append(os.path.abspath('..'))

from models import GatheringModel, read_input, compute_multiphase_pressure_drop
import pandas as pd
from gamspy import Options, Ord

In [2]:
m = GatheringModel("Prototipo", data= read_input("toy_problem.xlsx"))

--- Job _HE4OyOOsToCJA2wSpKraSA.gms Start 09/09/25 17:33:41 50.4.1 84b10359 WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\Users\Diego\Desktop\ExxonMobil\LM_Gathering\venv\Lib\site-packages\gamspy_base\gmsprmNT.txt
    C:\Users\Diego\Documents\GAMS\gamsconfig.yaml
--- GAMS Parameters defined
    Input C:\Users\Public\Documents\Wondershare\CreatorTemp\tmp9kc1bogq\_HE4OyOOsToCJA2wSpKraSA.gms
    Output C:\Users\Public\Documents\Wondershare\CreatorTemp\tmp9kc1bogq\_HE4OyOOsToCJA2wSpKraSA.lst
    ScrDir C:\Users\Public\Documents\Wondershare\CreatorTemp\tmp9kc1bogq\tmp5b3ixnq5\
    SysDir C:\Users\Diego\Desktop\ExxonMobil\LM_Gathering\venv\Lib\site-packages\gamspy_base\
    LogOption 3
    Trace C:\Users\Public\Documents\Wondershare\CreatorTemp\tmp9kc1bogq\_HE4OyOOsToCJA2wSpKraSA.txt
    License C:\Users\Diego\Documents\GAMSPy\gamspy_license.txt
    OptFile 0
    OptDir C:\Users\Public\Documents\Wondershare\CreatorTemp\tmp9kc1bogq\
    LimRow 0
    LimCol 0
    TraceOpt 3
    SolPrint 0


In [ ]:
m.solution_algorithm(scenario_name="toy_problem2", max_iterations=10, solver_opts={"solver": "gurobi", "gap": 0.0001, "max_time": 1000.0})

--- Iteration 1 ---
--- _HE4OyOOsToCJA2wSpKraSA.gms(1053) 11 Mb
--- Job _HE4OyOOsToCJA2wSpKraSA.gms Start 09/09/25 17:34:58 50.4.1 84b10359 WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\Users\Diego\Desktop\ExxonMobil\LM_Gathering\venv\Lib\site-packages\gamspy_base\gmsprmNT.txt
    C:\Users\Diego\Documents\GAMS\gamsconfig.yaml
--- GAMS Parameters defined
    MIQCP gurobi
    Input C:\Users\Public\Documents\Wondershare\CreatorTemp\tmp9kc1bogq\_HE4OyOOsToCJA2wSpKraSA.gms
    Output C:\Users\Public\Documents\Wondershare\CreatorTemp\tmp9kc1bogq\_HE4OyOOsToCJA2wSpKraSA.lst
    ScrDir C:\Users\Public\Documents\Wondershare\CreatorTemp\tmp9kc1bogq\tmp5b3ixnq5\
    SysDir C:\Users\Diego\Desktop\ExxonMobil\LM_Gathering\venv\Lib\site-packages\gamspy_base\
    LogOption 3
    Trace C:\Users\Public\Documents\Wondershare\CreatorTemp\tmp9kc1bogq\_HE4OyOOsToCJA2wSpKraSA.txt
    License C:\Users\Diego\Documents\GAMSPy\gamspy_license.txt
    OptFile 0
    OptDir C:\Users\Public\Documents\Wondershare\

[CONVERTER - INFO] GAMS (.gms) file has been generated under toy_problem/iteration_1\Multiphase_network_design.gms


--- _HE4OyOOsToCJA2wSpKraSA.gms(1123) 158 Mb
--- Job _HE4OyOOsToCJA2wSpKraSA.gms Start 09/09/25 17:37:11 50.4.1 84b10359 WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\Users\Diego\Desktop\ExxonMobil\LM_Gathering\venv\Lib\site-packages\gamspy_base\gmsprmNT.txt
    C:\Users\Diego\Documents\GAMS\gamsconfig.yaml
--- GAMS Parameters defined
    Input C:\Users\Public\Documents\Wondershare\CreatorTemp\tmp9kc1bogq\_HE4OyOOsToCJA2wSpKraSA.gms
    Output C:\Users\Public\Documents\Wondershare\CreatorTemp\tmp9kc1bogq\_HE4OyOOsToCJA2wSpKraSA.lst
    ScrDir C:\Users\Public\Documents\Wondershare\CreatorTemp\tmp9kc1bogq\tmp5b3ixnq5\
    SysDir C:\Users\Diego\Desktop\ExxonMobil\LM_Gathering\venv\Lib\site-packages\gamspy_base\
    LogOption 3
    Trace C:\Users\Public\Documents\Wondershare\CreatorTemp\tmp9kc1bogq\_HE4OyOOsToCJA2wSpKraSA.txt
    License C:\Users\Diego\Documents\GAMSPy\gamspy_license.txt
    OptFile 0
    OptDir C:\Users\Public\Documents\Wondershare\CreatorTemp\tmp9kc1bogq\
    LimRow 

[CONVERTER - INFO] GAMS (.gms) file has been generated under toy_problem/iteration_2\Multiphase_network_design.gms


--- _HE4OyOOsToCJA2wSpKraSA.gms(1247) 191 Mb
--- Job _HE4OyOOsToCJA2wSpKraSA.gms Start 09/09/25 17:40:33 50.4.1 84b10359 WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\Users\Diego\Desktop\ExxonMobil\LM_Gathering\venv\Lib\site-packages\gamspy_base\gmsprmNT.txt
    C:\Users\Diego\Documents\GAMS\gamsconfig.yaml
--- GAMS Parameters defined
    Input C:\Users\Public\Documents\Wondershare\CreatorTemp\tmp9kc1bogq\_HE4OyOOsToCJA2wSpKraSA.gms
    Output C:\Users\Public\Documents\Wondershare\CreatorTemp\tmp9kc1bogq\_HE4OyOOsToCJA2wSpKraSA.lst
    ScrDir C:\Users\Public\Documents\Wondershare\CreatorTemp\tmp9kc1bogq\tmp5b3ixnq5\
    SysDir C:\Users\Diego\Desktop\ExxonMobil\LM_Gathering\venv\Lib\site-packages\gamspy_base\
    LogOption 3
    Trace C:\Users\Public\Documents\Wondershare\CreatorTemp\tmp9kc1bogq\_HE4OyOOsToCJA2wSpKraSA.txt
    License C:\Users\Diego\Documents\GAMSPy\gamspy_license.txt
    OptFile 0
    OptDir C:\Users\Public\Documents\Wondershare\CreatorTemp\tmp9kc1bogq\
    LimRow 

[CONVERTER - INFO] GAMS (.gms) file has been generated under toy_problem/iteration_3\Multiphase_network_design.gms


--- _HE4OyOOsToCJA2wSpKraSA.gms(1371) 191 Mb
--- Job _HE4OyOOsToCJA2wSpKraSA.gms Start 09/09/25 17:42:40 50.4.1 84b10359 WEX-WEI x86 64bit/MS Windows
--- Applying:
    C:\Users\Diego\Desktop\ExxonMobil\LM_Gathering\venv\Lib\site-packages\gamspy_base\gmsprmNT.txt
    C:\Users\Diego\Documents\GAMS\gamsconfig.yaml
--- GAMS Parameters defined
    Input C:\Users\Public\Documents\Wondershare\CreatorTemp\tmp9kc1bogq\_HE4OyOOsToCJA2wSpKraSA.gms
    Output C:\Users\Public\Documents\Wondershare\CreatorTemp\tmp9kc1bogq\_HE4OyOOsToCJA2wSpKraSA.lst
    ScrDir C:\Users\Public\Documents\Wondershare\CreatorTemp\tmp9kc1bogq\tmp5b3ixnq5\
    SysDir C:\Users\Diego\Desktop\ExxonMobil\LM_Gathering\venv\Lib\site-packages\gamspy_base\
    LogOption 3
    Trace C:\Users\Public\Documents\Wondershare\CreatorTemp\tmp9kc1bogq\_HE4OyOOsToCJA2wSpKraSA.txt
    License C:\Users\Diego\Documents\GAMSPy\gamspy_license.txt
    OptFile 0
    OptDir C:\Users\Public\Documents\Wondershare\CreatorTemp\tmp9kc1bogq\
    LimRow 

(False, 3)